In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor

# Capstone Project - Function 4

## Function 4 - Fast, but Inaccurate Modelling

**Problem Description** This example is for a particular business relying heavily on online sales. It can run very accurate calculations to figure out what is the optimal placement of their product across warehouses. Unfortunately, the calculations are extremely expensive (computationally) to run, so they can only do it once every two weeks. Instead, they propose using a machine learning model which approximates the solution quickly (in a few hours). The model has four hyper-parameters you need to tune, and the output corresponds to the difference between the expensive calculation, and the model. Since you are modelling a dynamical system, expect a lot of local optima!

**Acquisition Function found to be the most efficient over time:** Expected Improvement

**Kernel Idea (evolved over time from simple default kernel usage to parametrized kernel based on inputs/outputs and function context):**
For a blackbox optimization problem like this, where we are dealing with a dynamical system with many local optima, a common choice is the Radial Basis Function (RBF) kernel, also known as the squared exponential kernel. This kernel is a good choice because it is flexible and can model a wide range of functions.

### 1.Loading the available data:

In [2]:
# Loading the baseline data
X = np.load('initial_data/function_4/initial_inputs.npy')
Y = np.load('initial_data/function_4/initial_outputs.npy')
X2 = np.load('initial_data2/function_4/initial_inputs.npy')
Y2 = np.load('initial_data2/function_4/initial_outputs.npy')

X = np.concatenate((X,X2), dtype = float)
Y = np.concatenate((Y,Y2), dtype = float)

# Precision notation
np.set_printoptions(suppress=False)

# Adding knwon inputs/outputs from previous weeks
Xn = np.array([
    [0.111111, 0.111111, 0.111111, 0.111111],
    [0.37931, 0.413793, 0.310344, 0.448275],
    [0.435897, 0.384615, 0.358974, 0.410256],
    [0.430897, 0.382615, 0.352974, 0.400256],
    [0.435897, 0.384615, 0.358974, 0.410256],
    [0.441176, 0.352941, 0.382352, 0.382352],
    [0.387755, 0.346938, 0.428571, 0.387755],
    [0.448979, 0.367346, 0.367346, 0.387755],
    [0.000001, 0.551020, 0.999999, 0.000001],
    [0.408163, 0.346938, 0.408163, 0.387755],
    [0.384615, 0.589743, 0.487179, 0.435897],
    [0.418918, 0.391891, 0.351351, 0.418918],
    [0.135135, 0.337837, 0.662162, 0.31081 ],
    [0.282051, 0.358974, 0.641025, 0.282051],
    [0.472973, 0.216216, 0.162162, 0.486486],
    [0.435897, 0.230769, 0.179487, 0.487179],
    [0.512821, 0.333333, 0.512821, 0.179487],
    [0.351351, 0.364864, 0.5,      0.364864],
    [0.489795, 0.081632, 0.387755, 0.306122],
    [0.448979, 0.265306, 0.408163, 0.32653 ]
    ])
X = np.concatenate((X,Xn), dtype = float)

Yn = np.array(
[
    -16.709552,
    -0.8783565419,
    0.4459034519,
    0.4593234589,
    0.4459034519,
    0.2039978146,
    0.3753309654,
    -0.02709952725,
    -28.79430339,
    0.4651345698,
    -4.324019297,
    0.6211206476,
    -9.177094288,
    -6.205305156,
    -7.043141706,
    -5.531307365,
    -5.031563033,
    -0.9773689956,
    -7.770325094,
    -1.65968593
])
Y = np.concatenate((Y,Yn), dtype = float)

In [3]:
print("X:\n")
print(X)
print("\n")
print("Y:\n")
print(Y)

X:

[[8.96981054e-01 7.25627970e-01 1.75404309e-01 7.01694369e-01]
 [8.89356396e-01 4.99587855e-01 5.39268858e-01 5.08783439e-01]
 [2.50946243e-01 3.36931305e-02 1.45380025e-01 4.94932421e-01]
 [3.46962061e-01 6.25040024e-03 7.60563606e-01 6.13023557e-01]
 [1.24871181e-01 1.29770193e-01 3.84400483e-01 2.87076101e-01]
 [8.01302707e-01 5.00231094e-01 7.06644560e-01 1.95102841e-01]
 [2.47708262e-01 6.04454273e-02 4.21863451e-02 4.41324251e-01]
 [7.46702242e-01 7.57091504e-01 3.69353060e-01 2.06566281e-01]
 [4.00665027e-01 7.25742511e-02 8.86768254e-01 2.43842290e-01]
 [6.26070596e-01 5.86751259e-01 4.38805782e-01 7.78857694e-01]
 [9.57135293e-01 5.97644383e-01 7.66113852e-01 7.76209905e-01]
 [7.32812426e-01 1.45249979e-01 4.76812718e-01 1.33365734e-01]
 [6.55115479e-01 7.23918269e-02 6.87151746e-01 8.15165642e-02]
 [2.19734429e-01 8.32031335e-01 4.82864162e-01 8.25692306e-02]
 [4.88594190e-01 2.11965096e-01 9.39177907e-01 3.76191726e-01]
 [1.67130486e-01 8.76554558e-01 2.17239545e-01 9.59

In [4]:
y_max = np.max(Y)
print("Max output until now: ", y_max, "which corresponds to input: ", X[np.where(Y == y_max)][0])

Max output until now:  0.6211206476 which corresponds to input:  [0.418918 0.391891 0.351351 0.418918]


### 2. Build a Bayesian Model
Here we will build the bayesian model using Gaussian Process Regression (GPR). This method was explored in the course and is also the method recommended by the litterature.
GPR is widely used in blackbox machine learning models due to its representation flexibility and inherent capability to quantify uncertainty over predictions.

Advantages:
- Flexibility: GPR is a non-parametric method, meaning it makes fewer prior assumptions about the form of the function that maps inputs to outputs.
- Uncertainty Quantification: One of the key advantages of GPR is its ability to provide a measure of uncertainty along with predictions.
- Kernel Functions: GPR uses kernel functions to define the similarity between data points. This allows the model to capture complex patterns and structures in the data. We played with kernel functions during the full capstone project to find the best candidate.
- Prior Knowledge Incorporation: GPR allows for the incorporation of prior knowledge through the use of different kernel functions.
- Surrogate Modeling: GPR is often used as a surrogate model in optimization problems, where the goal is to find the best parameters of a blackbox function that is expensive to evaluate. 

Inconvenients:
- Computation resources: It can be computationally expensive for large datasets, and choosing the right kernel function can be challenging.

#### Approach for Kernel and parameters
For a blackbox optimization problem like this, where we are dealing with a dynamical system with many local optima, a common choice is the Radial Basis Function (RBF) kernel, also known as the squared exponential kernel. This kernel is a good choice because it is flexible and can model a wide range of functions.

In [5]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define the kernel function
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1)

# Create a GaussianProcessRegressor object
gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)

# Fit to data using Maximum Likelihood Estimation of the parameters
gpr.fit(X, Y)

C:\Users\guill\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=0.1),
                         n_restarts_optimizer=11)

### 3. Acquisition Function 1 - Upper Confidence Bound

Upper-Confidence Bound action selection uses uncertainty in the action-value estimates for balancing exploration and exploitation. Since there is inherent uncertainty in the accuracy of the action-value estimates when we use a sampled set of rewards thus UCB uses uncertainty in the estimates to drive exploration.
We started the captstone with high exploration (ucb = mean + 1.96 * standard_deviation) but ended up with more exploitation (1.96 -> 1.12) when inputs were converging, still more exploration than Function 1 and 2 due to current outputs found.

In [6]:
import itertools as it

x_range = np.linspace(0, 1, 50)

# Create the 4D grid using NumPy
dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)

mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 0.90 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query: ", UB_NextQuery)

UCB - Next Query:  [0.67346939 0.48979592 0.40816327 0.        ]


### 4. Acquisition Function 2 - Probability of Improvement

The Probability of Improvement (PI) is an acquisition function which chooses the next query point as the one which has the highest probability of improvement over the current best observation. 
In other words, for each candidate point x, it assigns the probability of f(x) being larger than our current best f(x*). 
This helps balance the trade-off between exploration (probing areas of the search space where the outcome is uncertain) and exploitation (probing areas where the model predicts a good outcome).

In [7]:
from scipy.stats import norm

# Compute PI values for candidate points
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - y_max) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

# Choose the next point with maximum PI value
next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query: ", PI_NextQuery)

PI - Next Query:  [0.24489796 0.30612245 0.48979592 0.42857143]


## 5. Acquisition Function - Thompson Sampling

Thompson Sampling is a heuristic for decision-making problems that balances exploration and exploitation. It works by sampling parameters from the posterior distribution, then choosing the action that maximizes the expected reward.

In [9]:
x_range = np.linspace(0, 1, 40)

# Create the 4D grid using NumPy
dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)

# Compute Thompson Sampling values for candidate points
def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

# Choose the next point with maximum Thompson Sampling value
next_idx = np.argmax(thompson_values)
TS_NextQuery = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", TS_NextQuery)


Thompson Sampling - Next Query:  [0.         0.25641026 0.82051282 1.        ]


### 6. Acquisition Function - Bayesian Expected Losses

Bayesian Expected Loss is the expected value of a loss function with respect to the posterior distribution. It's used in decision-making to compare actions, where the preferred action is the one for which the posterior expected loss is smaller. This concept is central to Bayesian decision theory.

In [10]:
# Compute the expected loss for candidate points
def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - y_max) / sigma
    expected_loss = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

# We re-use X_grid due to computational needs
expected_loss_values = [compute_expected_loss(x) for x in X_grid]

# Choose the next point with minimum expected loss
next_idx = np.argmin(expected_loss_values)
BL_NextQuery = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", BL_NextQuery)

Bayesian Expected Loss - Next Query:  [0.43589744 0.23076923 0.17948718 0.48717949]


### 7. Acquisition Function - Expected Improvement
Expected Improvement (EI) is an acquisition function which selects the next point to sample by maximizing the expected improvement over the current best estimate. Unlike Probability of Improvement, EI considers not just the probability, but also the magnitude of the potential improvement.

In [5]:
from scipy.stats import norm

x_range = np.linspace(0, 1, 40)

# Create the 4D grid using NumPy
dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)


# Compute Expected Improvement for candidate points
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - y_max) / sigma
    ei = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

# Choose the next point with maximum EI value
next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)



KeyboardInterrupt



## Predict Output for next query
Based on the next queries found using our set of acquisition functions, we use the prediction of the model to see which prediction would maximizse the output (potentially) and send the top 1 or 2 each week to the form. 

In [14]:
formatted_UCB = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("UCB: ",UB_NextQuery[0],UB_NextQuery[1],UB_NextQuery[2],UB_NextQuery[3], gpr.predict([UB_NextQuery]))
formatted_PI = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("PI: ",PI_NextQuery[0],PI_NextQuery[1], PI_NextQuery[2], PI_NextQuery[3], gpr.predict([PI_NextQuery]))
#formatted_TS = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("TS: ",TS_NextQuery[0],TS_NextQuery[1], TS_NextQuery[2], TS_NextQuery[3], gpr.predict([TS_NextQuery]))
#formatted_BL = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("BL: ",BL_NextQuery[0],BL_NextQuery[1], BL_NextQuery[2], BL_NextQuery[3], gpr.predict([BL_NextQuery]))
formatted_EI = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("EI: ",EI_NextQuery[0],EI_NextQuery[1], EI_NextQuery[2], EI_NextQuery[3], gpr.predict([EI_NextQuery]))

print(formatted_UCB)
print(formatted_PI)
#print(formatted_TS)
#print(formatted_BL)
print(formatted_EI)

UCB: 0.000000-0.310811-0.405405-0.594595 -> [3.24115516]
PI: 0.459459-0.324324-0.459459-0.270270 -> [4.68123293]
TS: 0.000000-0.256410-0.820513-1.000000 -> [-0.72779519]
BL: 0.435897-0.230769-0.179487-0.487179 -> [-5.53131135]
EI: 0.512821-0.333333-0.512821-0.179487 -> [6.82806489]
